# Data Cleaning 

#### 1. Import pandas library.

In [2]:
import pandas as pd

#### 2. Import the users table.

In [3]:
users=pd.read_csv("../data/users.csv")

#### 3. Rename Id column to userId.

In [4]:
users.rename({"userId":"ID"},axis=1,inplace=True)
users.rename({"ID":"userId"},axis=1,inplace=True)
users.head()

,Unnamed: 0,userId,Reputation,Views,UpVotes,DownVotes
0,0,-1,1,0,5007,1920
1,1,2,101,25,3,0
2,2,3,101,22,19,0
3,3,4,101,11,0,0
4,4,5,6792,1145,662,5


#### 4. Import the posts table. 

In [5]:
posts=pd.read_csv("../data/posts.csv")
posts.head()

,Unnamed: 0,PostId,userId,Score,ViewCount,CommentCount
0,0,1,8.0,23,1278.0,1
1,1,2,24.0,22,8198.0,1
2,2,3,18.0,54,3613.0,4
3,3,4,23.0,13,5224.0,2
4,4,5,23.0,81,NaN,3


#### 5. Rename Id column to postId and OwnerUserId to userId.

In [6]:
posts.rename({"PostId":"ID","userId":"OwnerUserId"},axis=1,inplace=True)
posts.rename({"ID":"PostId","OwnerUserId":"userId"},axis=1,inplace=True)


#### 6. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

In [7]:
users_sliced=users.loc[:, "userId":"DownVotes"]
users_sliced


,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
...,...,...,...,...,...
40320,55743,1,0,0,0
40321,55744,6,1,0,0
40322,55745,101,0,0,0
40323,55746,106,1,0,0


In [8]:
posts_sliced=posts.loc[:,"PostId":"CommentCount"]
posts_sliced

,PostId,userId,Score,ViewCount,CommentCount
0,1,8.0,23,1278.0,1
1,2,24.0,22,8198.0,1
2,3,18.0,54,3613.0,4
3,4,23.0,13,5224.0,2
4,5,23.0,81,NaN,3
...,...,...,...,...,...
91971,115374,805.0,2,NaN,2
91972,115375,49365.0,0,9.0,0
91973,115376,55746.0,1,5.0,2
91974,115377,805.0,0,NaN,0


#### 7. Merge the two dataframes created in the step above (8), users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [22]:
merged=users_sliced.merge(posts_sliced,how="inner",on="userId")
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90584 entries, 0 to 90583
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   userId        90584 non-null  int64  
 1   Reputation    90584 non-null  int64  
 2   Views         90584 non-null  int64  
 3   UpVotes       90584 non-null  int64  
 4   DownVotes     90584 non-null  int64  
 5   PostId        90584 non-null  int64  
 6   Score         90584 non-null  int64  
 7   ViewCount     42188 non-null  float64
 8   CommentCount  90584 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 6.9 MB


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [23]:
merged.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
PostId              0
Score               0
ViewCount       48396
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

In [26]:
merged=merged.fillna(0)

In [27]:
merged.isnull().sum()

userId          0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
PostId          0
Score           0
ViewCount       0
CommentCount    0
dtype: int64

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [31]:
merged.dtypes

userId            int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
PostId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

In [ ]:
print("I guess ViewCount being the only float64 type and with NaN values in it")